In [77]:
import pandas as pd
import bs4 as bs
import requests
import json
import sys
import os
import re

In [2]:
def writeLinksJson(article, append=False):
    file = 'dataBase/articles.json'
    if append:
        with open(file, 'a+') as jsonOut:
            json.dump(article, jsonOut, indent=2, ensure_ascii=False)
    else:
        with open(file, 'w+') as jsonOut:
            json.dump(article, jsonOut, indent=2, ensure_ascii=False)
            # ensure_ascii=False : pour bien afficher les caractéres accentueux

In [3]:
def readLinksJson():
    file = 'dataBase/articles.json'
    if os.path.isfile(file):
        with open(file) as jsonIn:
            data = json.load(jsonIn)
            return data
    else:
        print("I can't find ", file)

In [4]:
def common(a, b):
    c = [value for value in a if value in b]
    return c

def common2(a, b):
    return set(a).intersection(b)

In [5]:
def articleLinkIdGenerator(online=False):
    # on a utilisé le online=false car si on est pas en ligne on pourra
    # utiliser la dataBase pour afficher les resultats voulue
    url = 'https://pagellapolitica.it/'
    uriPage = 'dichiarazioni/verificato?page='
    jsonFile = 'dataBase/articles.json'
    articles = {}
    if online:
        for i in range(sys.maxsize):
            pageNumber = i
            pageNumber = str(pageNumber)
            pageAddress = url + uriPage + pageNumber

            resp = requests.get(pageAddress)
            if resp:
                source = requests.get(pageAddress).text
                soup = bs.BeautifulSoup(source, "lxml")
                tables = soup.findAll('div', {'class': 'col-lg-3 mb-7'})
                for table in tables:
                    uriArticleID = table.find(
                        'a', {'class': 'statement-link'})['href']
                    idPattern = re.search(r'/([0-9]+)/', uriArticleID)
                    idNumber = idPattern.group(1)
                    articles[idNumber] = {
                        'url': url+uriArticleID,
                    }
                if idNumber == str(215):
                    break
            if idNumber == str(215):
                break

        with open(jsonFile, 'w') as file:
            # la sortie de Json est une variable ici c'est a
            json.dump(articles, file, indent=2, ensure_ascii=False)
    else:
        with open(jsonFile) as file:
            articles = json.load(file)
    return articles

In [31]:
def extractInfoArticle(id, online=False):

    article = {}
    url = 'https://pagellapolitica.it/'
    if online:
        uriArticle = 'dichiarazioni/'
        address = url + uriArticle + id
        #print(address)
        resp = requests.get(address)
        if resp:
            source = requests.get(address).text
            soup = bs.BeautifulSoup(source, "lxml")

            # auteur
            p = soup.find(
                'p', {'class': 'h4 mb-1 px-2 text-dark font-weight-light'})
            author = p.find('a', {'class': 'u-link-muted'}).text.strip()

            # le titre au complet quand on rentre dans l'article
            divFullBody = soup.find('div', {'class': 'col-lg-9 mb-9 mb-lg-0'})
            if divFullBody:
                # veracity
                # h2 mark-ni
                # h2 mark-vero
                # h2 mark-ceri
                # h2 mark-pinocchio
                # h2 mark-panzana

                veracities = ["mark-ni", "mark-vero",
                              "mark-ceri", "mark-pinocchio", "mark-panzana"]
                divTitle = divFullBody.find('div', {'class': 'mb-2 mt-2 px-2'})
                divTitlePH2Px2 = divFullBody.find(
                    'p', {'class': 'h2 px-2'})
                if divTitle:
                    fullTitle = divTitle.find(
                        'span', {'class': 'h2'}).text.strip()

                    spanVeracity = divTitle.find(
                        'span', {'class': 'h2'})
                    thisVeracity = spanVeracity['class']
                    veracity = common(thisVeracity, veracities)[0]
                elif divTitlePH2Px2:
                    fullTitle = divTitlePH2Px2.find(
                        'span', {'class': 'text-darker'}).text.strip()
                    spanVeracity = divTitlePH2Px2.find(
                        'span', {'class': 'text-darker'})
                    thisVeracity = spanVeracity['class']
                    veracity = common(thisVeracity, veracities)[0]

                else:
                    # dans les anciens artictles le fullText n'existe pas donc
                    # si il ne le trouve pas il affiche le msg
                    fullTitle = "No fullTitle"
                    veracity = "No veracity"

            # les claims
            pclaimPX3 = soup.find(
                'p', {'class': 'lead px-3 py-3 bg-light g-brd-around g-brd-lightblue'})
            pclaimPX2 = soup.find('p', {'class': 'h2 px-2'})
            if pclaimPX3:
                claim = pclaimPX3.text.strip()
            elif pclaimPX2:
                claim = pclaimPX2.find(
                    'span', {'class': 'text-darker'}).text.strip()

            # les dates(publication,origine) elles sont dans la méme classe d'ou l'utilisation
            # de divDates[0] pour la publication et divDates[1] pour l'origine
            divDate = soup.find('div', {'class': 'card-body pt-0 px-2'})
            divDates = divDate.findAll('div', {'class': 'col-lg-2 text-left'})
            datePublished = divDates[0].find(
                'span', {'class': 'text-dark'}).text.strip()
            dateOrigin = divDates[1].find(
                'span', {'class': 'text-dark'}).text.strip()

            # statement source
            divReferredLinks = divDate.find('div', {'class': 'col-lg-4'})
            statementSource = divReferredLinks.find(
                'a', {'class': 'u-link-muted'})['href']

            # le join est pour concaténé les textes des balises P pour les mettre en un seul txt
            divMainArticle = soup.find('div', {'id': 'analisi-content'})
            listOfPInMainArticle = divMainArticle.findAll('p')
            mainText = [text.text.strip() for text in listOfPInMainArticle]
            mainArticle = ' '.join(mainText)

            # les liens de reference dans le main text
            listOfAllLinks = []
            for link in divMainArticle.findAll('a', href=True):
                #print(link['href'])
                listOfAllLinks.append(link)
                #if link['href']:
                    #print("link:",link['href'])
            listOfAllLinksHref = [link['href'] for link in listOfAllLinks if link['href']]

            # Tags
            if divFullBody.find('div', {'class': 'px-2 u-space-2-top'}):
                divTags = divFullBody.find(
                    'div', {'class': 'px-2 u-space-2-top'})
                tags = divTags.findAll(
                    'a', {'class': 'btn btn-sm btn-light u-btn-light transition-3d-hover rounded-0 mb-2'})

                tagList = []
                for tag in tags:
                    tagList.append(tag.text.strip())

    else:
        data = readLinksJson()
        # pour l'ajout d'un element dans un dictionnaire exsitant ,**dic[2] recupére
        # tout les anciens elements du dic
        data[id] = {
            **data[id],
            # le site de fact checking
            "source": "source not found",
            "claim": "claim not found",
            "body": "mainArticle not found",                     # le text de l'article
                    "referred_links": "listOfAllLinksHref not found",    # tous les liens dans le texte
                    "title": "fullTitle not found",                      # le titre de l'article
                    "date": "dateOrigin not found",                      # date de la claim
                    "url": "address not found",  # url de l'article
                    "tags": "tags not found",                          # les mots cles
                    "author": "author not found",                       # auteur de la claim
                    "datePublished": "datePublished not found",
                    "rating_value": "rating_value not found",          # la valeur de la veracite
                    "statementSource": "statementSource not found",
                    # les entities nomes qui est extraite de la claim
                    "claim_entities": "claim_entities not found",
                    # les entities nomes qui est extraite de l'article
                    "body_entities": "body_entities not found",
                    # parmi les tages, les entities nomes a partir de la tag
                    "keyword_entities": "keyword_entities not found",
                    # les entities nomes a partir de l'auteur de la claim
                    "author_entities": "author_entities not found",
                    "review_author": "review_author not found"         # l'auteur de l'article

        }

    article = {
        id: {
            "source": url,                           # le site de fact checking
            "claim": claim,
            "body": mainArticle,                     # le text de l'article
            "referred_links": listOfAllLinksHref,    # tous les liens dans le texte
            "title": fullTitle,                      # le titre de l'article
            "date": dateOrigin,                      # date de la claim
            "url": address,  # url de l'article
            "tags": tagList,                          # les mots cles
            "author": author,                       # auteur de la claim
            "datePublished": datePublished,
            "rating_value": veracity,          # la valeur de la veracite
            "statementSource": statementSource,
            # les entities nomes qui est extraite de la claim
            "claim_entities": "claim_entities not found",
            # les entities nomes qui est extraite de l'article
            "body_entities": "body_entities not found",
            # parmi les tages, les entities nomes a partir de la tag
            "keyword_entities": "keyword_entities not found",
            # les entities nomes a partir de l'auteur de la claim
            "author_entities": "author_entities not found",
            "review_author": "review_author not found"         # l'auteur de l'article

        }
    }

    return article

# Extraction of all data in an article

In [40]:
article = extractInfoArticle("1327", online=True)
for link in article["1327"]["referred_links"]:
    print(link)
    
#print(article)

http://www.quirinale.it/qrnw/statico/costituzione/costituzione.htm#Principi
http://www.salute.gov.it/ministero/sezMinistero.jsp?label=ssn
http://www.regione.piemonte.it/pianosanitario/dwd/piano_socio_sanitario.pdf
http://www.salute.gov.it/portale/news/p3_2_1_1_1.jsp?lingua=italiano&menu=notizie&p=dalministero&id=936
http://www.parlamento.it/parlam/leggi/deleghe/98112dl.htm


# Get a list of article links by page number

In [73]:
def pageLinks(pageNumber):
    url = 'https://pagellapolitica.it/'
    uri = 'dichiarazioni/verificato?page='
    pageAddress = url + uri + pageNumber
    resp = requests.get(pageAddress)
    if resp:
        source = requests.get(pageAddress).text
        soup = bs.BeautifulSoup(source, "lxml")
        tables = soup.findAll('div', {'class': 'col-lg-3 mb-7'})
        pageLinks = []
        for table in tables:
            link = table.find('a', {'class': 'statement-link'})['href']
            articleLink = url+link
            pageLinks.append(articleLink)
    return pageLinks

In [53]:
pageLinks = pageLinks("1")
for i in pageLinks:
    print(i)

https://pagellapolitica.it/dichiarazioni/8595/si-il-mes-non-puo-prendere-decisioni-senza-il-voto-dellitalia
https://pagellapolitica.it/dichiarazioni/8594/quali-promesse-non-sono-state-mantenute-per-il-ponte-di-genova
https://pagellapolitica.it/dichiarazioni/8593/si-litalia-ha-gia-un-extra-deficit-da-80-miliardi-di-euro
https://pagellapolitica.it/dichiarazioni/8592/salvini-ha-ragione-sui-furbi-che-non-rispettano-la-quarantena
https://pagellapolitica.it/dichiarazioni/8591/no-non-sono-gia-disponibili-540-miliardi-di-euro-grazie-allue
https://pagellapolitica.it/dichiarazioni/8590/no-non-riaprono-i-musei-mentre-le-messe-rimangono-vietate
https://pagellapolitica.it/dichiarazioni/8589/no-lemilia-romagna-non-ha-il-primato-di-morti-per-covid-19-nelle-rsa
https://pagellapolitica.it/dichiarazioni/8588/di-maio-sbaglia-sul-funzionamento-di-immuni-la-app-di-contact-tracing
https://pagellapolitica.it/dichiarazioni/8587/debito-perpetuo-e-a-tasso-zero-e-davvero-possibile
https://pagellapolitica.it/dich

# Extract id of each article by ReGex

In [43]:
def idNumber(link):
    idPattern = re.search(r'/([0-9]+)/', link)
    idNumber = idPattern.group(1)
    return idNumber

# Testing extractInfoArticle() and idNumber()

At first execute the method pageLinks(pageNumber)

In [72]:
#for pageNumber in range(42, 174):
pageLinks = pageLinks("3")
for page in pageLinks:
    id = idNumber(page)
    lastArticle = extractInfoArticle(id, online=True)
    #ter.writeLinksJson(lastArticle, append=True)
    #print('claim = ', lastArticle[id]["claim"])
    print('URL = ', lastArticle[id]["url"])
    print('rating_value = ', lastArticle[id]["rating_value"])
    print("\n")

URL =  https://pagellapolitica.it/dichiarazioni/8561
rating_value =  mark-ni


URL =  https://pagellapolitica.it/dichiarazioni/8560
rating_value =  mark-pinocchio


URL =  https://pagellapolitica.it/dichiarazioni/8559
rating_value =  mark-vero


URL =  https://pagellapolitica.it/dichiarazioni/8558
rating_value =  mark-vero


URL =  https://pagellapolitica.it/dichiarazioni/8557
rating_value =  mark-pinocchio


URL =  https://pagellapolitica.it/dichiarazioni/8556
rating_value =  mark-ni


URL =  https://pagellapolitica.it/dichiarazioni/8555
rating_value =  mark-ni


URL =  https://pagellapolitica.it/dichiarazioni/8554
rating_value =  mark-ceri


URL =  https://pagellapolitica.it/dichiarazioni/8553
rating_value =  mark-ceri


URL =  https://pagellapolitica.it/dichiarazioni/8552
rating_value =  mark-panzana


URL =  https://pagellapolitica.it/dichiarazioni/8551
rating_value =  mark-ceri


URL =  https://pagellapolitica.it/dichiarazioni/8550
rating_value =  mark-ceri


URL =  https://pagell

At first execute the method pageLinks(pageNumber)

In [74]:
def findPageNumberOfID(id):
    for pageNumber in range(1, 200):
        print("Looking in page", pageNumber)
        pageURLs = pageLinks(str(pageNumber))
        for page in pageURLs:
            idWanted = idNumber(page)
            if idWanted == id:
                print("ID", id, "is in the page:", pageNumber)
                break
        if idWanted == id:
            break

In [75]:
findPageNumberOfID("8552")

Looking in page 1
Looking in page 2
Looking in page 3
ID 8552 is in the page: 3


In [99]:
def dataFrameOfIDs(articlesIDs):
    data = {}
    for id in articlesID:
        currentArticle = extractInfoArticle(id, online=True)
        data.update(currentArticle)
    df = pd.DataFrame(data)
    return df

In [100]:
articlesIDs = ["8547","8546","8549","8555"]
dataFrameOfIDs(articlesIDs)


,8547,8546,8549,8555
source,https://pagellapolitica.it/,https://pagellapolitica.it/,https://pagellapolitica.it/,https://pagellapolitica.it/
claim,«I moduli di autosegnalazione dei pugliesi rie...,«Con i nostri governi eravamo ad ospitare 50 m...,«Tutti devono sapere: Germania padrona dell’Ue...,«Io preferisco guardare solo ai nuovi contagia...
body,"Il 9 marzo mattina, ospite di Agorà su Rai3, i...","Il 9 marzo, in un’intervista a La Repubblica, ...","Il senatore Gianluigi Paragone, espulso dal M5...",Il 24 marzo il deputato di Italia Viva Luigi M...
referred_links,[https://twitter.com/agorarai/status/123692837...,[https://rep.repubblica.it/ws/detail/intervist...,[https://www.facebook.com/watch/?v=10434821560...,[https://twitter.com/marattin/status/124251297...
title,Emiliano impreciso sui pugliesi militari e su ...,Renzi esagera sui detenuti nelle carceri itali...,"No, la Germania per il coronavirus non mette 2...",Marattin fa un po’ di confusione sul calo dei ...
date,09 mar 2020,10 mar 2020,14 mar 2020,24 mar 2020
url,https://pagellapolitica.it/dichiarazioni/8547,https://pagellapolitica.it/dichiarazioni/8546,https://pagellapolitica.it/dichiarazioni/8549,https://pagellapolitica.it/dichiarazioni/8555
tags,"[Michele Emiliano, Militari, Coronavirus, pugl...","[Giustizia, Carceri, Italia Viva]","[Germania, Coronavirus, paragone, aiuti imprese]","[Italia Viva, Coronavirus, protezione civile]"
author,Michele Emiliano,Matteo Renzi,Gianluigi Paragone,Luigi Marattin
datePublished,10 mar 2020,10 mar 2020,17 mar 2020,25 mar 2020
